In [ ]:
# wip.006

## Update
- Use UpSample2d+Conv2d instead of ConvTrans2D, see comments for detail
- Add Batch Normalization layer after each conv
- You can use crf method (https://www.kaggle.com/meaninglesslives/apply-crf) to improve the result 
## Changelog
- Changed uncov to uconv, but removed the dropout in the last layer
- Corrected sanity check of predicted validation data (changed from ids_train to ids_valid)
- Used correct mask (from original train_df) for threshold tuning (inserted y_valid_ori)

# About
Since I am new to learning from image segmentation and kaggle in general I want to share my noteook.
I saw it is similar to others as it uses the U-net approach. I want to share it anyway because:

- As said, the field is new to me so I am open to suggestions.
- It visualizes some of the steps, e.g. scaling, to learn if the methods do what I expect which might be useful to others (I call them sanity checks).
- Added stratification by the amount of salt contained in the image.
- Added augmentation by flipping the images along the y axes (thanks to the forum for clarification).
- Added dropout to the model which seems to improve performance.

In [ ]:
from __future__ import division
import numpy as np
import pandas as pd

from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize
from skimage.transform import rotate

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook


#import pydensecrf.densecrf as dcrf
from skimage.color import gray2rgb
from skimage.color import rgb2gray
from skimage.io import imread, imsave, imshow
#from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral


# Params and helpers

In [ ]:
img_size_ori = 101
img_size_target = 128

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]
    
def img_mask_plot(img,mask):
    fix, axs = plt.subplots(1, 2, figsize=(10,10))
    axs[0].imshow(img.squeeze(), cmap="Greys")
    axs[0].set_title("Image")

    axs[1].imshow(mask.squeeze(), cmap="Greys")
    axs[1].set_title("Mask")
    
def img_3_plot(img1,img2,img3,tit="",tits=["Img1","Img2","Img3"]):
    fix, axs = plt.subplots(1, 3, figsize=(8,8))
    fix.suptitle(tit,fontsize=10)
    axs[0].imshow(img1.squeeze(), cmap="Greys")
    axs[0].set_title(tits[0])
    axs[1].imshow(img2.squeeze(), cmap="Greys")
    axs[1].set_title(tits[1])
    axs[2].imshow(img3.squeeze(), cmap="Greys")
    axs[2].set_title(tits[2])
    fix.subplots_adjust(top=1.5)
    plt.show()

def img_4_plot(img1,img2,img3,img4,tit="",tits=["Img1","Img2","Img3","Img4"]):
    fix, axs = plt.subplots(1, 4, figsize=(8,8))
    fix.suptitle(tit,fontsize=10)
    axs[0].imshow(img1.squeeze(), cmap="Greys")
    axs[0].set_title(tits[0])
    axs[1].imshow(img2.squeeze(), cmap="Greys")
    axs[1].set_title(tits[1])
    axs[2].imshow(img3.squeeze(), cmap="Greys")
    axs[2].set_title(tits[2])
    axs[3].imshow(img4.squeeze(), cmap="Greys")
    axs[3].set_title(tits[3])
    fix.subplots_adjust(top=1.5)
    plt.show()

def img_5_plot(img1,img2,img3,img4,img5,tit="",tits=["Img1","Img2","Img3","Img4","Img5"]):
    fix, axs = plt.subplots(1, 5, figsize=(8,8))
    fix.suptitle(tit,fontsize=10)
    axs[0].imshow(img1.squeeze(), cmap="Greys")
    axs[0].set_title(tits[0])
    axs[1].imshow(img2.squeeze(), cmap="Greys")
    axs[1].set_title(tits[1])
    axs[2].imshow(img3.squeeze(), cmap="Greys")
    axs[2].set_title(tits[2])
    axs[3].imshow(img4.squeeze(), cmap="Greys")
    axs[3].set_title(tits[3])
    axs[4].imshow(img5.squeeze(), cmap="Greys")
    axs[4].set_title(tits[4])
    fix.subplots_adjust(top=1.5)
    plt.show()

    
def iou_clean_old(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return iou

def iou_clean(y_true_in, y_pred_in):
    intr = np.sum((y_true_in+y_pred_in)==2)
    unio = np.sum(y_true_in+y_pred_in)-intr
    if unio == 0:
        ret = 1
    else:
        ret = intr/unio
    return ret

def prec_clean(y_true_in, y_pred_in):
    tp = np.sum((y_true_in+y_pred_in)==2)
    fp = np.sum(y_true_in == (y_pred_in+1))
    fn = np.sum(y_true_in == (y_pred_in-1))
    sum_p = tp+fp+fn
    if sum_p == 0:
        pr = 1
    else:
        pr = tp/sum_p
    return pr
    

def iou_clean_batch(y_true_in, y_pred_in):
    import numpy as np
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_clean(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.array(metric)

def prec_clean_batch(y_true_in, y_pred_in):
    import numpy as np
    batch_size = y_true_in.shape[0]
    prec = []
    for batch in range(batch_size):
        value = prec_clean(y_true_in[batch], y_pred_in[batch])
        prec.append(value)
    return np.array(prec)


def error_estimation_batch(true_mask,pred_mask):
    ious = iou_clean_batch(true_mask,pred_mask)
    precs= prec_clean_batch(true_mask,pred_mask)
    coefs = np.zeros(ious.shape)
    coefs[ious>=0.50] = 0.1
    coefs[ious>=0.55] = 0.2
    coefs[ious>=0.60] = 0.3
    coefs[ious>=0.65] = 0.4
    coefs[ious>=0.70] = 0.5
    coefs[ious>=0.75] = 0.6
    coefs[ious>=0.80] = 0.7
    coefs[ious>=0.85] = 0.8
    coefs[ious>=0.90] = 0.9
    coefs[ious>=0.95] = 1.0
    value = np.mean(precs*coefs)
    return value

# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [ ]:
"""
Function which returns the labelled image after applying CRF

"""
#Original_image = Image which has to labelled
#Mask image = Which has been labelled by some technique..
def crf(original_image, mask_img, gt_prob=0.7, steps = 10):
    
    # Converting annotated image to RGB if it is Gray scale
    if(len(mask_img.shape)<3):
        mask_img = gray2rgb(mask_img)

#     #Converting the annotations RGB color to single 32 bit integer
#    annotated_label = mask_img[:,:,0] + (mask_img[:,:,1]<<8) + (mask_img[:,:,2]<<16)
    annotated_label = mask_img[:,:,0] + (np.int64(mask_img[:,:,1])<<8) + (np.int64(mask_img[:,:,2])<<16)

    
#     # Convert the 32bit integer color to 0,1, 2, ... labels.
    colors, labels = np.unique(annotated_label, return_inverse=True)

    n_labels = 2
    
    #Setting up the CRF model
    d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

    # get unary potentials (neg log probability)
    U = unary_from_labels(labels, n_labels, gt_prob, zero_unsure=False)
    d.setUnaryEnergy(U)

    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
                      normalization=dcrf.NORMALIZE_SYMMETRIC)
        
    #Run Inference for (10) steps 
    Q = d.inference(steps)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    return MAP.reshape((original_image.shape[0],original_image.shape[1]))

In [ ]:
# Image transformations:
def warp_an_image(im_in,n1=6,n2=6):
#    from skimage.transform import PiecewiseAffineTransform, warp, resize
#    import skimage.util

    rows,cols = im_in.shape[0], im_in.shape[1]
    im_in_rows = np.linspace(0, cols, 20)
    im_in_cols = np.linspace(0, rows, 10)
    src_rows, src_cols = np.meshgrid(im_in_rows, im_in_cols)

    src = np.dstack([src_cols.flat, src_rows.flat])[0]

    # add sinusoidal oscillation to row coordinates:
    dst_rows = src[:, 1] - np.sin(np.linspace(0, n2 * np.pi, src.shape[0])) * n1
    dst_cols = src[:, 0]

    #dst_rows *= 1.
    #dst_rows -= 1. * n1
    dst = np.vstack([dst_cols, dst_rows]).T

    tform = PiecewiseAffineTransform()
    tform.estimate(src, dst)

    out_rows = im_in.shape[0]# - 1. * n1
    out_cols = cols
    out = warp(im_in, tform, output_shape=(out_rows, out_cols))
    out = out[15:85,15:85]
    out = resize(out,(128,128))
    
    return out

def get_a_tform(im_in,n1=6,n2=6):
    from skimage.transform import PiecewiseAffineTransform, warp, resize
    import skimage.util

    rows,cols = im_in.shape[0], im_in.shape[1]
    im_in_rows = np.linspace(0, cols, 20)
    im_in_cols = np.linspace(0, rows, 10)
    src_rows, src_cols = np.meshgrid(im_in_rows, im_in_cols)

    src = np.dstack([src_cols.flat, src_rows.flat])[0]

    # add sinusoidal oscillation to row coordinates:
    dst_rows = src[:, 1] - np.sin(np.linspace(0, n2 * np.pi, src.shape[0])) * n1
    dst_cols = src[:, 0]

    #dst_rows *= 1.
    #dst_rows -= 1. * n1
    dst = np.vstack([dst_cols, dst_rows]).T

    tform = PiecewiseAffineTransform()
    tform.estimate(src, dst)
    return tform
    
def warp_an_image_fast(im_in,tfrm,out_cols,out_rows):
    from skimage.transform import warp
    out = warp(im_in, tform, output_shape=(out_rows, out_cols))
    out = out[15:85,15:85]
    out = resize(out,(128,128))
    return out    
    
def swirl_an_image(im_in):
    from skimage.transform import swirl,resize
    out = swirl(im_in,strength=2)
    out = resize(out,(128,128))
    return out

# Loading of training/testing ids and depths
Reading the training data and the depths, store them in a DataFrame. Also create a test DataFrame with entries from depth not in train.

In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

# Read images and masks
Load the images and masks into the DataFrame and divide the pixel values by 255.

In [ ]:
train_df["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

# Calculating the salt coverage and salt coverage classes
Counting the number of salt pixels in the masks and dividing them by the image size. Also create 11 coverage classes, -0.1 having no salt at all to 1.0 being salt only.
Plotting the distribution of coverages and coverage classes, and the class against the raw coverage.

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

In [ ]:
n_cl = 10
def cov_to_class(val):    
    for i in range(0, 1+n_cl):
        if val * n_cl <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=n_cl, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
plt.scatter(train_df.coverage, train_df.coverage_class)
plt.xlabel("Coverage")
plt.ylabel("Coverage class")

# Plotting the depth distributions
Separatelty plotting the depth distributions for the training and the testing data.

In [ ]:
sns.distplot(train_df.z, label="Train")
sns.distplot(test_df.z, label="Test")
plt.legend()
plt.title("Depth distribution")

# Show some example images

In [ ]:
max_images = 20
grid_width = 10
plot_class = 1
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
#for i, idx in enumerate(train_df[train_df.coverage_class==plot_class].index[:max_images]):
for i, idx in enumerate(train_df.index[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Reds")
    ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
    ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
    ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

# Create train/validation split stratified by salt coverage
Using the salt coverage as a stratification criterion. Also show an image to check for correct upsampling.

In [ ]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_valid, depth_train, depth_valid = train_test_split(
    train_df.index.values,
    np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state=1337)

In [ ]:
# Save the Split information
pd.DataFrame(ids_train,columns=["ids_train"]).to_csv("ids_train.csv")
pd.DataFrame(ids_valid,columns=["ids_valid"]).to_csv("ids_valid.csv")

In [ ]:
# Load the Split information and re-build the data from train_df
ids_train = np.array(pd.read_csv("ids_train.csv").ids_train)
ids_valid = np.array(pd.read_csv("ids_valid.csv").ids_valid)

x_train = np.array(train_df.loc[ids_train]["images"].map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)
y_train = np.array(train_df.loc[ids_train]["masks"].map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)
cov_train = train_df[train_df.index.isin(ids_train)]["coverage"].values
depth_train = train_df[train_df.index.isin(ids_train)]["z"].values

x_valid = np.array(train_df.loc[ids_valid]["images"].map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)
y_valid = np.array(train_df.loc[ids_valid]["masks"].map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)
cov_valid = train_df[train_df.index.isin(ids_valid)]["coverage"].values
depth_valid = train_df[train_df.index.isin(ids_valid)]["z"].values


In [ ]:
tmp_idx = 30
tmp_img = np.zeros((img_size_target, img_size_target), dtype=train_df.images.loc[ids_train[tmp_idx]].dtype)
tmp_img[:img_size_ori, :img_size_ori] = train_df.images.loc[ids_train[tmp_idx]]


fix, axs = plt.subplots(1, 2, figsize=(15,5))
axs[0].imshow(tmp_img, cmap="Greys")
axs[0].set_title("Original image")

axs[1].imshow(x_train[tmp_idx].squeeze(), cmap="Greys")
axs[1].set_title("Upsampled image")

# Build model

In [ ]:
def conv_block(m, dim, acti, bn, res, do=0):
	n = Conv2D(dim, 3, activation=acti, padding='same')(m)
	n = BatchNormalization()(n) if bn else n
	n = Dropout(do)(n) if do else n
	n = Conv2D(dim, 3, activation=acti, padding='same')(n)
	n = BatchNormalization()(n) if bn else n
	return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
	if depth > 0:
		n = conv_block(m, dim, acti, bn, res)
		m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
		m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
		if up:
			m = UpSampling2D()(m)
			m = Conv2D(dim, 2, activation=acti, padding='same')(m)
		else:
			m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
		n = Concatenate()([n, m])
		m = conv_block(n, dim, acti, bn, res)
	else:
		m = conv_block(m, dim, acti, bn, res, do)
	return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
		 dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
	i = Input(shape=img_shape)
	o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
	o = Conv2D(out_ch, 1, activation='sigmoid')(o)
	return Model(inputs=i, outputs=o)

In [ ]:
model = UNet((img_size_target,img_size_target,1),start_ch=16,depth=5,batchnorm=True)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

# Data augmentation

In [ ]:
x_train_bak = x_train.copy()
y_train_bak = y_train.copy()
depth_train_back = depth_train.copy()
cov_train_back = cov_train.copy()

In [ ]:
# Augment with left-right flips
x_train = np.append(x_train, [np.fliplr(y) for y in x_train_bak], axis=0)
y_train = np.append(y_train, [np.fliplr(y) for y in y_train_bak], axis=0)
depth_train = np.append(depth_train,depth_train_back,axis=0)
cov_train = np.append(cov_train,cov_train_back,axis=0)

In [ ]:
# Augment with warp
from skimage.transform import PiecewiseAffineTransform, warp, resize
import skimage.util

x_train = np.append(x_train, [warp_an_image(y) for y in x_train_bak], axis=0)
y_train = np.append(y_train, [warp_an_image(y) for y in y_train_bak], axis=0)
depth_train = np.append(depth_train,depth_train_back,axis=0)
cov_train = np.append(cov_train,cov_train_back,axis=0)

In [ ]:
x_train_bak[0].shape

In [ ]:
x_train_bak.shape

In [ ]:
y_train.shape

In [ ]:
fig, axs = plt.subplots(3, 10, figsize=(15,3))
for i in range(10):
    axs[0][i].imshow(x_train[i].squeeze(), cmap="Blues")
    axs[0][i].imshow(y_train[i].squeeze(), cmap="Reds", alpha=0.3)
    axs[1][i].imshow(x_train[int(len(x_train_bak) + i)].squeeze(), cmap="Blues")
    axs[1][i].imshow(y_train[int(len(y_train_bak) + i)].squeeze(), cmap="Reds", alpha=0.3)
    axs[2][i].imshow(x_train[2*int(len(x_train_bak)) +i].squeeze(), cmap="Blues")
    axs[2][i].imshow(y_train[2*int(len(y_train_bak)) +i].squeeze(), cmap="Reds", alpha=0.3)
fig.suptitle("Top row: original images, bottom row: augmented images")

# Training

In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.wip.005.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

epochs = 200
batch_size = 32

history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr],shuffle=True)

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax_acc.plot(history.epoch, history.history["acc"], label="Train accuracy")
ax_acc.plot(history.epoch, history.history["val_acc"], label="Validation accuracy")

In [ ]:
model = load_model("../submissions/0.737/keras.wip.005.model")
#model = load_model("../submissions/0.758/keras.model")

# Predict the masks, use Validation 1 dataset

In [ ]:
preds_valid = model.predict(x_valid).reshape(-1, img_size_target, img_size_target)
preds_valid = np.array([downsample(x) for x in preds_valid])
y_valid_ori = np.array([train_df.loc[idx].masks for idx in ids_valid])

In [ ]:
# Plot some random images
for val_idx in range(4):
    img_3_plot(x_valid[val_idx],preds_valid[val_idx],y_valid[val_idx],tit="val_idx="+str(val_idx),\
              tits=["image","prediction","truth"])

In [ ]:
# Plot more images
max_images = 60
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(ids_valid[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    pred = preds_valid[i]
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.imshow(pred, alpha=0.3, cmap="OrRd")
    ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
    ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
    ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

# Determine the best 0/1 threshold, use Validation 1 dataset
Use the contest metrics against the validation set

In [ ]:
thr_array = []
err_array = []
ious_array  = []
for thr in np.arange(0.0,1.0,0.005):
    pred_mask_thr = np.array(np.round(preds_valid > thr), dtype=np.float32)
    err = error_estimation_batch(true_mask=y_valid_ori, pred_mask=pred_mask_thr)
    ious = iou_clean_batch(y_pred_in=pred_mask_thr,y_true_in=y_valid_ori)
    cov_pred = np.sum(np.sum(pred_mask_thr,axis=1),axis=1)/101**2
    low_ious_cnt = np.sum(ious<0.5)
    low_ious_cnt_low_cov = np.sum(np.logical_and(ious<0.5,cov_pred<0.5))
    print(thr,err,low_ious_cnt,low_ious_cnt_low_cov)
    thr_array.append(thr)
    err_array.append(err)
    ious_array.append(ious) 



In [ ]:
# Option 1:
# Fixed threshold:
threshold_best = thr_array[np.argmax(err_array)]
err_best       = err_array[np.argmax(err_array)]
iou_best       = ious_array[np.argmax(err_array)]
pred_mask_thr = np.array(np.round(preds_valid > 1.0*threshold_best), dtype=np.float32)
err = error_estimation_batch(true_mask=y_valid_ori, pred_mask=pred_mask_thr)
cov_pred = np.sum(np.sum(pred_mask_thr,axis=1),axis=1)/101**2

print("threshold_best: ",threshold_best)
print("error_best: ",err_best)
print("err: ",err)

In [ ]:
# Option 2:
# Vector threshold:
# Build the variable vector threshold:
threshold_best_array = np.full(preds_valid.shape[0],threshold_best)
# Override the best threshold for low salt coverage predictions:
cov_pred = np.sum(np.sum(pred_mask_thr,axis=1),axis=1)/101**2
threshold_best_array[cov_pred<0.5] = 0.6

# Buid the mask: 
pred_mask_thr = [preds_valid[x] > threshold_best_array[x] for x in range(preds_valid.shape[0])]

# Re-calculate the error
err = error_estimation_batch(true_mask=y_valid_ori, pred_mask=pred_mask_thr)
err


In [ ]:
plt.plot(thr_array, err_array)
plt.plot(threshold_best, err_best, "sr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
#plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

In [ ]:
# IoU with threshold:
ious = iou_clean_batch(y_pred_in=pred_mask_thr,y_true_in=y_valid_ori)

In [ ]:
ious.shape,depth_valid.shape

In [ ]:
cov_pred = np.sum(np.sum(pred_mask_thr,axis=1),axis=1)/101**2

In [ ]:
# IoU vs. depth
plt.plot(depth_valid,ious,'db')

In [ ]:
# IoU vs. salt coverage
plt.plot(cov_valid,ious,'db')

In [ ]:
plt.plot(cov_valid,ious,'db')

In [ ]:
plt.plot(cov_pred,cov_valid,'db')

In [ ]:
np.sum(ious<=0.1)

In [ ]:
for val_idx in np.where(ious<=0.1)[0]:
    img_4_plot(x_valid[val_idx],preds_valid[val_idx],pred_mask_thr[val_idx],y_valid[val_idx],\
               tit="val_idx="+str(val_idx),tits=["img","pred","pred_thr","true"])

# Full Prediction and Error Evaluation, Validation 1 dataset

In [ ]:
preds_valid = model.predict(x_valid).reshape(-1, img_size_target, img_size_target)
preds_valid = np.array([downsample(x) for x in preds_valid])
y_valid_ori = np.array([train_df.loc[idx].masks for idx in ids_valid])
pred_mask_thr = np.array(np.round(preds_valid > threshold_best), dtype=np.float32)
err = error_estimation_batch(true_mask=y_valid_ori, pred_mask=pred_mask_thr)
print("error=",err)


# Full Prediction and Error Evaluation, Validation 2 dataset

In [ ]:
preds_valid_2 = model.predict(x_valid_2).reshape(-1, img_size_target, img_size_target)
preds_valid_2 = np.array([downsample(x) for x in preds_valid_2])
y_valid_ori_2 = np.empty((0,101,101))
y_valid_ori_2 = np.append(y_valid_ori_2,[rotate(y,30) for y in y_valid_ori],axis=0)
pred_mask_thr_2 = np.array(np.round(preds_valid_2 > threshold_best), dtype=np.float32)
err_2 = error_estimation_batch(true_mask=downsample(y_valid_ori_2), pred_mask=pred_mask_thr_2)
print(err_2)

# Have a look in the 2nd validation dataset - if available

In [ ]:
for val_idx in range(50):
    img_4_plot(x_valid_2[val_idx],preds_valid_2[val_idx],pred_mask_thr_2[val_idx],y_valid_2[val_idx],tit="val_idx="+str(val_idx),\
              tits=["img","prediction","prediction_thr","truth"])

# Adjusted threshold
Again some sample images with the adjusted threshold.

In [ ]:
max_images = 30
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(ids_valid[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    pred = preds_valid[i]
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.imshow(np.array(np.round(pred > threshold_best), dtype=np.float32), alpha=0.3, cmap="OrRd")
    ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
    ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
    ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

# Post-Processing
CRF

In [ ]:
# Some visualisation:
for val_idx in range(0,20):
    img1 = x_valid[val_idx][:,:,0]
    img1 = np.int64(np.round(img1*255))
    img2 = upsample(preds_valid[val_idx])
    #img3 = np.round(img2 > threshold_best)
    img3 = np.round(upsample(pred_mask_thr[val_idx]))
    img4 = y_valid[val_idx][:,:,0]
    img5 = crf(img1,img3,gt_prob=0.66)
    img_5_plot(img1,img2,img3,img4,img5,tit="val_idx="+str(val_idx),\
               tits=["img","pred_mask","pred_mask_thresh","true_mask","pred_mask_crf"])

In [ ]:
for steps in np.arange(8,15,1):
    for gt in np.arange(0.66,0.74,0.02):
        pred_mask_thr_crf = []
        for y in range(len(ids_valid)):
            img1 = downsample(x_valid[y][:,:,0])
            img1 = np.int64(np.round(img1*255))
            img2 = preds_valid[y]
            img3 = np.round((pred_mask_thr[y]))
            pred_mask_thr_crf.append((crf(img1,img3,gt_prob=gt,steps=steps)))
        print(steps,gt,":",error_estimation_batch(true_mask=y_valid_ori, pred_mask=pred_mask_thr_crf))

In [ ]:
gt_best = 0.68
steps_best = 11

# Validation Submission
Create a validation submission file

In [ ]:
preds_valid = model.predict(x_valid).reshape(-1, img_size_target, img_size_target)
valid_dict  = {idx: RLenc(np.round(downsample(preds_valid[i]) > threshold_best)) for i, idx in enumerate(tqdm_notebook(test_df.index.values))}
sub = pd.DataFrame.from_dict(valid_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.valid.wip.005.csv')

# Validation Post-Processing
CRF

# Submission
Load, predict and submit the test image predictions.

In [ ]:
x_test = np.array([upsample(np.array(load_img("../input/test/images/{}.png".format(idx), grayscale=True))) / 255 for idx in tqdm_notebook(test_df.index)]).reshape(-1, img_size_target, img_size_target, 1)

In [ ]:
preds_test = model.predict(x_test)

In [ ]:
pred_dict = {idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) for i, idx in enumerate(tqdm_notebook(test_df.index.values))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.wip.005.csv')